# Extending Fermi

In this guide, we will implement a simple RHF algorithm using tools from Fermi. Moreover, we will explore the composability of the code by integrating this new implementation into Fermi without ever touching its source.

## Goals

1. Implement a Restricted Hartree-Fock method
2. Integrate our implementation into Fermi
3. Use new code within Fermi

# Procedure

## Integrals

We will use Fermi to obtain the following equations over a basis set $\{\chi_1, \chi_2, ..., \chi_N\}$

Overlap: $ S_{\mu \nu} = \int \chi_\mu \chi_\nu d\tau$ 

Kinetic: $ T_{\mu \nu} = -\frac{1}{2}\int \chi_\mu \nabla^2 \chi_\nu d\tau$

Nuclear: $ V_{\mu \nu} = \sum_A \int \chi_\mu \frac{Z_A}{|\mathbf{r}-\mathbf{R_A}|} \chi_\nu d\tau$

Repulsion: $ (\chi_\mu \chi_\nu|\chi_\rho \chi_\sigma) = \int \chi_\mu(1) \chi_\nu(1)\frac{1}{r_{12}}\chi_\rho(2)\chi_\sigma(2)d\tau  \rangle$

## Working Equations

One-electron term: $H_{\mu\nu} = T_{\mu\nu} + V_{\mu\nu}$

Density Matrix: $D_{\mu\nu} = \sum_{i=1}^{n_{occ}} C_{\mu i} C_{\nu i}$

> $C$ contains the expansion coefficients of the orbitals $\phi$ with respect to the basis set:
> $\phi_{i} = \sum_{\mu} C_{\mu i}\chi_\mu$. Thus, this is array is essentially the wave function.

$G_{\mu\nu\rho\sigma} = 2(\chi_\mu \chi_\nu | \chi_\rho \chi_\sigma) - (\chi_\mu \chi_\rho | \chi_\nu \chi_\sigma)$

Fock matrix: $F_{\mu\nu} = H_{\mu\nu} + \sum_{\sigma \rho}D_{\sigma\rho}G_{\mu\nu\rho\sigma}$

HF Energy: $E = \sum_{\mu\nu} (2H_{\mu\nu} +  \sum_{\sigma \rho}D_{\sigma\rho}G_{\mu\nu\rho\sigma})D_{\nu\mu} + V_{nuc}$

## Initial Set up

1. Generate all integrals
2. Form the orthogonalizer $X = S^{-1/2}$
3. Set $C_{\mu\nu} = 0$ as initial guess

## Iterations

1. Build Fock Matrix
2. Transform the Fock matrix as: $\tilde{F} = XFX$
3. Diagonalize the transformed Fock, eigenvalues are orbitals energies and eigenvector $\tilde{C}_{\mu i}$ are the orbital coefficients
4. Backtransform orbital coefficients $C = X\tilde{C}$
5. Evaluate energy and check for convergeny 
6. If not converged, start over

# Part 1: Writting a RHF Code

Following the procedure above, we first need integrals. For that, we set the molecule options we want and create an integral manager:

In [30]:
# Load Fermi
using Fermi

# Load Fermi Integrals (just so we don't need to type Fermi.Integrals all the time)
using Fermi.Integrals

# We will also need linear algebra tools later on
using LinearAlgebra

@molecule {
    O    1.209153654800    1.766411818900   -0.017161397200
    H    2.198480007500    1.797710062700    0.012116171900
    H    0.919788188200    2.458018557000    0.629793883200
}

@set {
    basis sto-3g
    charge 0
    multiplicity 1 # Note that multiplicity must be one for RHF
}

# It is important to specify `eri_type`. The default will return a Sparse Array.
# The RHF algorithm is faster using this sparse array, but it also gets more complicated
# Here, we are looking for the simplest implementation
aoints = IntegralHelper(eri_type=Chonky())

 ⇒ Fermi IntegralHelper
 ⋅ Data Type:                 Float64
 ⋅ Basis:                     sto-3g
 ⋅ ERI:                       Chonky
 ⋅ Orbitals:                  AtomicOrbitals
 ⋅ Stored Integrals:          

> ⚠️ The integral helper returns `FermiArray` objects. Some libraries may not work well with those. You can retrieve the native Julia array of `X` *via* `X.data` or `Array(X)`

Now we can write a function that takes this integral object in and returns the HF energy

> If you never coded RHF youself, it may be beneficial to try it first before reading the code below

In [73]:
function MyRHF(aoints)

    # Get integrals
    println("Collecting Integrals")
    S = aoints["S"]
    T = aoints["T"]
    V = aoints["V"]
    H = T + V
    G = 2*aoints["ERI"] - permutedims(aoints["ERI"], (1,3,2,4))
    X = S^(-1/2)
      
    # Get nuclear repulsion
    Vnuc = aoints.molecule.Vnuc
    
    # Get the number of doubly occupied orbitals
    ndocc = aoints.molecule.Nα
    
    # Get the number of basis functions
    nbf = size(S, 1)
    
    # Create an array for C and set it to zero
    C = zeros(nbf, nbf)
    
    # Get density matrix
    D = C[:,1:ndocc] * (C[:,1:ndocc])'
    
    # Starts iterations!
    ΔE = 1.0 # arbitrary, just to start the loop
    Eold = 0.0
    Enew = 0.0
    ϵ = zeros(nbf)
    
    println("Starting Iterations!")
    
    while ΔE > 1e-8 

        Eold = Enew
        
        # Build Fock matrix
        F = similar(H)
        F .= H # Don't do F = H !!
        
        for μ = 1:nbf
            for ν = 1:nbf
                for ρ = 1:nbf
                    for σ = 1:nbf
                        F[μ,ν] += G[μ,ν,ρ,σ]*D[σ,ρ]
                    end
                end
            end
        end
        
        # Tarsnform F
        tF = X'*F*X
        
        # Diagonalize F
        ϵ, tC = diagonalize(tF)
        
        # Backtransform C
        C = X*tC
        
        # Update density matrix
        D = C[:,1:ndocc] * (C[:,1:ndocc])'
        
        # Compute energy
        Enew = Vnuc
        for μ = 1:nbf
            for ν = 1:nbf
                Enew += 2*H[μ,ν]*D[μ,ν] # Watch out! This portion cannot be all the way inside the loop
                for ρ = 1:nbf
                    for σ = 1:nbf
                        Enew += G[μ,ν,ρ,σ]*D[σ,ρ]*D[μ,ν]
                    end
                end
            end
        end
              
        # Compute ΔE
        ΔE = abs(Enew - Eold)
        
        # Print some msg!
        println("New energy: $Enew  - ΔE = $ΔE")
    end
    
    # Return energy and orbitals
    return Enew, ϵ, C
end

MyRHF (generic function with 1 method)

Now we can run the code!

> ⚠️ There is a lot of room for improvement here. This code was meant to be as simple and readable as possible. If you are learning about these methods it can be a good exercise to optimize the code above.

In [74]:
MyRHF(aoints)

Starting Iterations!
New energy: -73.24872578289293  - ΔE = 73.24872578289293
New energy: -74.93626077860303  - ΔE = 1.6875349957100951
New energy: -74.96369421293538  - ΔE = 0.02743343433235168
New energy: -74.96483372693868  - ΔE = 0.0011395140032988138
New energy: -74.96497219792273  - ΔE = 0.0001384709840550613
New energy: -74.96499697363893  - ΔE = 2.4775716198632836e-5
New energy: -74.965001733572  - ΔE = 4.759933062814525e-6
New energy: -74.96500266590847  - ΔE = 9.32336476466844e-7
New energy: -74.96500284954517  - ΔE = 1.836367005125794e-7
New energy: -74.96500288577498  - ΔE = 3.622980671025289e-8
New energy: -74.96500289292607  - ΔE = 7.151086833800946e-9


(-74.96500289292607, [-20.24593804071889, -1.2522378591086847, -0.600793616076149, -0.44902579654800745, -0.38938307081421075, 0.5733893104295433, 0.703982076890003], [0.9942071930373049 -0.2341333825561707 … 0.12766385894163132 5.598499071952242e-12; 0.025891913606589677 0.845738063406074 … -0.8288578348980302 -4.0005834130573896e-11; … ; -0.005614048428499817 0.15615844012365226 … 0.7740251002995622 0.8097737106988028; -0.005614048428377685 0.15615844011998248 … 0.7740251003544538 -0.8097737106324205])

# Part 2: Integrating with Fermi

We now move on to make our new code part of Fermi. We use Julia's multiple dispath system to extend Fermi functions without having to touch its source code.

We must, however, understand how methods are called inside Fermi. For a RHF computation, the function `Fermi.HartreeFock.RHF` is called. Different RHF implementations are called depending on a special argument used to select the algorithm. This argument is a subtype of `Fermi.HartreeFock.RHFAlgorithm`. For example:

if we create 

`alg = Fermi.HartreeFock.RHFa()`

and then run

`Fermi.HartreeFock.RHF(alg)`

The implementation associated with `RHFa` is gonna be used.

Thus, the first step is to create a algorithm object for our new code.

In [57]:
struct MyRHFAlg <: Fermi.HartreeFock.RHFAlgorithm end

At this point, running `Fermi.HartreeFock.RHF(MyRHFAlg())` will raise an error. We need to import this function and create a new method for it.

In [61]:
# Explicitly import the function
import Fermi.HartreeFock: RHF

# Create a new method of the function with our implementation
function RHF(alg::MyRHFAlg)
    aoints = IntegralHelper(eri_type=Chonky())
    MyRHF(aoints)
end

RHF

Now, our code is recognized in Fermi! We can create the algorithm object

`alg = MyRHFAlg()`

and call

`Fermi.HartreeFock.RHF(alg)`

or use the macro syntax

`@energy alg => rhf`

In [64]:
alg = MyRHFAlg()
@energy alg => rhf

Starting Iterations!
New energy: -73.24872578289293  - ΔE = 73.24872578289293
New energy: -74.93626077860303  - ΔE = 1.6875349957100951
New energy: -74.96369421293538  - ΔE = 0.02743343433235168
New energy: -74.96483372693868  - ΔE = 0.0011395140032988138
New energy: -74.96497219792273  - ΔE = 0.0001384709840550613
New energy: -74.96499697363893  - ΔE = 2.4775716198632836e-5
New energy: -74.965001733572  - ΔE = 4.759933062814525e-6
New energy: -74.96500266590847  - ΔE = 9.32336476466844e-7
New energy: -74.96500284954517  - ΔE = 1.836367005125794e-7
New energy: -74.96500288577498  - ΔE = 3.622980671025289e-8
New energy: -74.96500289292607  - ΔE = 7.151086833800946e-9


(-74.96500289292607, [0.9942071930373049 -0.2341333825561707 … 0.12766385894163132 5.598499071952242e-12; 0.025891913606589677 0.845738063406074 … -0.8288578348980302 -4.0005834130573896e-11; … ; -0.005614048428499817 0.15615844012365226 … 0.7740251002995622 0.8097737106988028; -0.005614048428377685 0.15615844011998248 … 0.7740251003544538 -0.8097737106324205])

It would be nice to get ride of the first line `alg = MyRHFAlg()`. In the options, we can set the desired algorithm for rhf using `@set rhf_alg N`. Where `N` is an integer associated with an implementation. The function `Fermi.HartreeFock.get_rhf_alg` takes this number `N` and returns the algorithm object. This is done using *dispatch by value*, where a new method is defined for each different argument value.

Let us teach Fermi that the number 2 will correspond to our new implementation.

In [66]:
# Again, import the function explicitly
import Fermi.HartreeFock: get_rhf_alg

function get_rhf_alg(x::Val{2})
    MyRHFAlg()
end

get_rhf_alg (generic function with 4 methods)

Notice how these functions can now select the two algorithms.

In [67]:
get_rhf_alg(1)

Fermi.HartreeFock.RHFa()

In [68]:
get_rhf_alg(2)

MyRHFAlg()

We can now set `rhf_alg` to 2 and run our code.

In [70]:
@molecule {
    He 0.0 0.0 0.0
    He 1.0 0.0 0.0 
}

@set {
    basis 6-31g
    rhf_alg 2
}

@energy rhf

Starting Iterations!
New energy: -5.219362055405596  - ΔE = 5.219362055405596
New energy: -5.549560170418672  - ΔE = 0.3301981150130757
New energy: -5.551082299776921  - ΔE = 0.0015221293582490247
New energy: -5.551087932737347  - ΔE = 5.632960426105171e-6
New energy: -5.551087974147649  - ΔE = 4.141030185422778e-8
New energy: -5.551087974974068  - ΔE = 8.26418933286277e-10


(-5.551087974974068, [-0.4076825876353103 -0.4581967915973321 0.7963788744123831 0.8454407693047463; -0.2748222783812665 -0.5432821451618882 -1.2631789382863847 -0.7221912389356774; -0.4076825876353115 0.4581967915973311 -0.7963788744123738 0.8454407693047553; -0.2748222783812684 0.5432821451618886 1.263178938286377 -0.7221912389356915])

# *Fully* Integrating our code

Up to this point we have been a little careless with the return of our function. In Fermi, our methods are expected to return wave function objects. `RHF` functions should return `RHF` objects. In order to create this object, we just need to call the constructor with all appropriate arguments.

In [79]:
function RHF(alg::MyRHFAlg)
    aoints = IntegralHelper(eri_type=Chonky())
    energy, eps, C = MyRHF(aoints)
    
    
    # 1st argument: molecule object
    molecule = aoints.molecule
    
    # 2nd argument: energy (got it above)
    # 3rd argument: Number of doubly occ orbitals
    ndocc = molecule.Nα
    # 4th argument: Number of virtual orbitals
    nvir = size(C,1) - ndocc
    # 5th argument: RHFOrbitals object
    orbitals = Fermi.Orbitals.RHFOrbitals(molecule, aoints.basis, eps, energy, C)
    # 6th and 7th, convergency parameters. We will skip those for now.
    
    return Fermi.HartreeFock.RHF(molecule, energy, ndocc, nvir, orbitals, 0.0, 0.0)
end

RHF

In [80]:
@energy rhf

Starting Iterations!
New energy: -5.219362055405596  - ΔE = 5.219362055405596
New energy: -5.549560170418672  - ΔE = 0.3301981150130757
New energy: -5.551082299776921  - ΔE = 0.0015221293582490247
New energy: -5.551087932737347  - ΔE = 5.632960426105171e-6
New energy: -5.551087974147649  - ΔE = 4.141030185422778e-8
New energy: -5.551087974974068  - ΔE = 8.26418933286277e-10


 ⇒ Fermi Restricted Hartree--Fock Wave function
 ⋅ Basis:                  6-31g
 ⋅ Energy:                 -5.551087974974068
 ⋅ Occ. Spatial Orbitals:  2
 ⋅ Vir. Spatial Orbitals:  2
Convergence: ΔE => 0.00e+00 Dᵣₘₛ => 0.00e+00

Now that our post-computation data is organized we can use this new RHF implementation for correlated computations!

In [82]:
wfn = @energy rhf
@energy wfn => mp2

Starting Iterations!
New energy: -5.219362055405596  - ΔE = 5.219362055405596
New energy: -5.549560170418672  - ΔE = 0.3301981150130757
New energy: -5.551082299776921  - ΔE = 0.0015221293582490247
New energy: -5.551087932737347  - ΔE = 5.632960426105171e-6
New energy: -5.551087974147649  - ΔE = 4.141030185422778e-8
New energy: -5.551087974974068  - ΔE = 8.26418933286277e-10
|                      Møller-Plesset Perturbation Theory                      |
|                                  Module  by                                  |
|                         G.J.R Aroeira and M.M. Davis                         |
  Starting MP2 computation
 Number of frozen orbitals:             0
 Number of inactive orbitals:           0
 Number of correlated electron pairs:   2
 Number of correlated virtual orbitals: 2
 ⇒ Total number of MP2 amplitudes:      16
--------------------------------------------------------------------------------
 Computing MP2 Energy... Done in 0.03635 s

   @Final RMP2 Co

 ⇒ Fermi Restricted MP2 Wave function
 ⋅ Correlation Energy:     -0.022365304453117275
 ⋅ Total Energy:           -5.573453279427185